In [1]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00


All API requests should include your API key in an Authorization HTTP header as follows:

```
Authorization: Bearer OPENAI_API_KEY
```



In [2]:
OPENAI_API_KEY = ###

In [6]:
import base64
import requests
import numpy as np
from skimage import img_as_ubyte, io
from PIL import Image
import io as io_module  # 메모리 버퍼를 위한 io 모듈

# 이미지를 base64로 인코딩하는 함수
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# white_patch 함수 정의 (화이트 밸런싱)
def white_patch(image, percentile=100):
    """
    Parameters
    ----------
    image : 입력으로 받는 이미지는 RGB 채널을 각각 가지는 (height, width, channels) 3차원 numpy 배열
    percentile : 채널 값의 보정값으로 사용하고자 하는 값을 지정하기 위한 비율, 기본값은 100으로 최대값을 사용
    """
    white_patch_image = img_as_ubyte((
                                      image * 1.0          # 이미지 배열을 부동 소수점 형식으로 변환하여 나눗셈 연산이 제대로 수행되도록 함
        							                / np.percentile(    # 지정된 백분위 수 계산
                                      	  image, 	            # 입력 이미지
                                     	    percentile,           # 입력 퍼센티지
                                     	    axis=(0, 1)           # 높이, 너비 축을 따라 계산
                                          )                # 이미지가 반환된 보정값을 사용해 정규화 됨
                                      ).clip(0, 1)         # 0,1 사이의 범위로 맞춰줌
                                     )
    return white_patch_image

# 이미지 전처리 함수
def img_preprocess(image_path):
    # 이미지를 불러온 후 화이트 밸런싱 적용
    input_image = io.imread(image_path)
    white_balanced_image = white_patch(input_image)

    pil_image = Image.fromarray(white_balanced_image)

    # 메모리 버퍼에 저장
    buffered = io_module.BytesIO()
    pil_image.save(buffered, format="PNG")

    # base64로 인코딩
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

# API 요청 및 응답 처리 함수
def process_image_and_initial_question(image_path, api_key):
    """
    Parameters
    ----------
    image_path : 이미지 파일의 경로
    api_key : OpenAI API 키
    """
    # 고정된 질문
    initial_question = "아기 기저귀 사진의 똥 부분만 보고 건강 상태를 설명해줘. 이상이 있다면 가능한 원인과 추가로 확인해야 할 상황에 대해 알려줘."

    # 이미지 전처리 및 base64 인코딩
    base64_image = img_preprocess(image_path)

    # 대화 기록을 저장할 리스트
    conversation_history = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": initial_question
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                        "detail": "auto"
                    }
                }
            ]
        }
    ]

    # 헤더 생성
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    # 첫 질문 보내기
    ask_question(conversation_history, headers)

    return conversation_history

# API 요청을 보내는 함수
def ask_question(conversation_history, headers, question=None):
    """
    Parameters
    ----------
    conversation_history : 대화 기록이 저장된 리스트
    headers : API 요청 헤더
    question : 추가 질문이 있을 경우 (옵션)
    """
    if question:
        conversation_history.append({
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": question
                }
            ]
        })

    # 요청할 payload 생성
    payload = {
        "model": "gpt-4o",
        "messages": conversation_history,
        "max_tokens": 300
    }

    # GPT-4 API로 요청 보내기
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # 응답 처리
    if response.status_code == 200:
        result = response.json()
        # 응답 내용 대화 기록에 추가
        conversation_history.append({
            "role": "assistant",
            "content": result['choices'][0]['message']['content']
        })
        print(result['choices'][0]['message']['content'])
    else:
        print(f"요청이 실패했습니다. 상태 코드: {response.status_code}, 오류 메시지: {response.text}")

# 대화 흐름 제어 함수
def conversation_flow(image_path, api_key):
    """
    Parameters
    ----------
    image_path : 이미지 파일의 경로
    api_key : OpenAI API 키
    """
    # 고정된 질문에 대해 첫 질문 보내기
    conversation_history = process_image_and_initial_question(image_path, api_key)

    # 추가 질문을 처리하는 루프
    while True:
        additional_question = input("추가적인 질문을 입력하세요 (종료하려면 'exit' 입력): ")
        if additional_question.lower() == 'exit':
            print("대화를 종료합니다.")
            break
        ask_question(conversation_history, headers, additional_question)

In [7]:
normal_6_img_url = '/content/drive/MyDrive/images_0601/normal/normal_6.JPG'
conversation_flow(normal_6_img_url, api_key)

사진을 기반으로 아기의 변 상태를 분석해 보겠습니다.

1. **색상**:
   - 노란색 변: 보통 모유 수유 중인 아기에게서 흔히 볼 수 있는 색상입니다.
   - 신생아 및 모유 수유 중인 아기의 변이 노란색, 겨자 색상, 또는 밝은 녹색을 띠는 것은 일반적입니다.

2. **질감**:
   - 사진의 변은 묽고 약간 점도가 있는 것처럼 보입니다.
   - 모유 수유하는 아기의 변은 일반적으로 묽거나 약간 걸쭉한 상태를 가질 수 있습니다.

### 이상의 징후와 원인:

1. **수분 변**: 묽은 변이나 물 같은 변이 반복적으로 나타나면 설사일 수 있습니다. 설사는 탈수를 유발할 수 있기 때문에 주의를 기울여야 합니다.
   - 설사의 가능한 원인: 
     - 세균이나 바이러스 감염 (예: 로타바이러스)
     - 알레르기나 음식 불내증
     - 항생제 등의 약물 부작용

2. **변의 색 변동**: 변이 녹색을 띤다면 너무 많은 포뮬러를 먹였거나(포뮬러 수유 아기의 경우) 음식 소화 문제가 있을 수 있습니다
추가적인 질문을 입력하세요 (종료하려면 'exit' 입력): 포뮬러가 뭐야?
포뮬러(Formula)는 영어로 '유아용 분유'를 의미합니다. 이는 모유 수유가 어려운 경우나 추가적인 영양 공급이 필요할 때 아기에게 제공되는 우유 대용 제품입니다. 분유는 아기에게 필요한 필수 영양소를 제공하도록 제조됩니다. 아기의 연령, 성장 단계 및 특정 요구 사항에 맞춰 다양한 종류의 분유가 있으며, 이는 대부분 우유나 콩 단백질을 기반으로 합니다.
추가적인 질문을 입력하세요 (종료하려면 'exit' 입력): exit
대화를 종료합니다.


In [8]:
diarrhea_4_img_url = '/content/drive/MyDrive/images_0601/diarrhea/diarrhea_4.jpeg'
conversation_flow(diarrhea_4_img_url, api_key)

사진 속 아기 기저귀의 대변은 황금색이며 점액질이 포함된 묽은 상태로 보입니다. 이런 식의 대변은 모유 수유를 하는 아기에게 흔히 나타나는 정상적인 모습일 수 있습니다. 그러나, 몇 가지 상황에서 우려할 필요가 있을 수 있습니다.

1. **정상적인 이유:**
   - 모유를 먹는 아기: 모유 수유 시 아기의 대변은 보통 황금색이나 초록색을 띠며 묽을 수 있습니다.
   - 소화되지 않은 음식: 이유식을 시작한 아기라면 새로운 음식이 소화되지 않아 이런 대변이 나올 수 있습니다.

2. **우려할 수 있는 이유:**
   - **감염:** 점액질이 많고 변 상태가 많이 묽다면 장염 등의 바이러스나 세균 감염 가능성을 고려할 수 있습니다.
   - **알레르기 반응:** 우유 단백질 또는 특정 음식에 대한 알레르기 반응으로 이러한 대변이 나올 수 있습니다.
   - **소화 문제:** 지속적인 점액질, 피 또는 아주 악취가 강한 대변은 소화 문제가 있을 수 있습니다.

3. **추가로 확인해야 할 상황:**
   - 아기가 무기력하거나 탈수 증상을 보이는지 (잦은 소변, 입과
추가적인 질문을 입력하세요 (종료하려면 'exit' 입력): 아기가 며칠 째 이런 똥을 싸고 있어
아기가 며칠째 계속해서 황금색의 묽은 대변을 보고 있다면 몇 가지 중요한 조치를 확인하고 검토해야 합니다. 

1. **증상 관찰:**
   - **탈수 증상:** 아기가 탈수 증상을 보이는지 확인하십시오. 입 주위가 마르거나 눈물이 나지 않거나 기저귀 교환 빈도가 줄어들 수 있습니다.
   - **체중 감소:** 아기의 체중이 줄거나 체중이 증가하지 않는 경우.
   - **기타 증상:** 발열, 무기력, 과도한 울음 등 다른 이상 증상이 있는지 확인하세요.

2. **먹는 것:** 
   - 새로운 음식: 아기가 새로운 음식을 먹기 시작했는지 확인하십시오. 새로운 음식이 원인일 수도 있습니다.
   - 수유 변화: 모유나 분유 교체가 있었는지, 또는 수유량의 극적인 변화가 있었는지 확인하세